In [ ]:
import os
import tensorflow as tf
import numpy as np #adds support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import pandas as pd #data structures and operations for manipulating numerical tables and time series
import sys # system path io
import glob #glob module is used to retrieve files/pathnames matching a specified pattern
from tqdm import tqdm # show progress bar when a loop is running
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly import tools
from plotly.graph_objs import *
from plotly.graph_objs.layout import Margin, YAxis, XAxis
init_notebook_mode()
import matplotlib.pyplot as plt
from matplotlib import patches as patches
from pathlib import Path
import xml.etree.ElementTree as ET
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.applications.resnet import ResNet50, preprocess_input
from sklearn.model_selection import KFold, StratifiedKFold
from keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input
from keras.applications.xception import Xception,preprocess_input, decode_predictions
from keras.applications.mobilenet import MobileNet,preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
import io
from google.colab import drive
from sklearn.model_selection import KFold, StratifiedKFold,LeaveOneOut
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.callbacks import EarlyStopping
import shutil
import statistics
from keras.models import Model
from tensorflow.keras import layers
drive.mount('/content/gdrive')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Structure: Diagnosis, Triage_Level
#NEED TO CODE FOR THE ZERO TRIAGE SCORE!
#Not real scores!
LookUp_Dictionary  = {
  'burns1':1,
  'burns2': 2,
  'burns3':10 ,
  'BacterialInfections': 5, 
  'NailDiseases':2 ,
  'ContactDermatitis':6,
  'UrticariaHives':3,
  'Others':0,
  'wounds': 7,
  'ViralInfections':5
}
print(LookUp_Dictionary)

{'burns1': 1, 'burns2': 2, 'burns3': 10, 'BacterialInfections': 5, 'NailDiseases': 2, 'ContactDermatitis': 6, 'UrticariaHives': 3, 'Others': 0, 'wounds': 7, 'ViralInfections': 5}


In [ ]:
IMAGE_SIZE=[250,250]
batch_size=32
NN  =  Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)  
#MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)   
for layer in NN.layers:
    layer.trainable = False
# layers creation - Here more can be added
x = layers.Flatten()(NN.output)
x = layers.Dense(1000, activation='relu')(x)
prediction = layers.Dense(10, activation='softmax')(x)
# creating a model object
model = Model(inputs=NN.input, outputs=prediction)
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
  loss='categorical_crossentropy',

  optimizer='sgd',

  metrics=['accuracy'] 

)

83697664/83683744 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [ ]:
import PIL
X=[]
Y=[]
base_dir="/content/gdrive/MyDrive/MBZUAI/Smart Triage/Potential Datasets/SkinBurns"
labels_list=['burns2','burns1','burns3','BacterialInfections', 'NailDiseases','ContactDermatitis','UrticariaHives','Others','wounds','ViralInfections']
dir=os.getcwd()
for directory in labels_list:
    print(directory)
    #print(len(os.listdir(base_dir+'/'+directory)))
    for files in os.listdir(base_dir+'/'+directory):
        #img = Image.open(directory+'/'+files)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
        X.append(directory+'/'+files)
        #X.append([np.array(img)])
        current_index=labels_list.index(directory)
        Y.append(current_index)
X=np.asarray(X)
Y=np.asarray(Y)

burns2
burns1
burns3
BacterialInfections
NailDiseases
ContactDermatitis
UrticariaHives
Others
wounds
ViralInfections


In [ ]:
!mkdir  validation
!mkdir  validation/burns2
!mkdir  validation/burns1
!mkdir  validation/burns3
!mkdir  validation/BacterialInfections
!mkdir  validation/NailDiseases
!mkdir  validation/ContactDermatitis
!mkdir  validation/UrticariaHives
!mkdir  validation/Others
!mkdir  validation/wounds
!mkdir  validation/ViralInfections

In [ ]:
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []

validation_path="validation"
train_path= '/content/gdrive/MyDrive/MBZUAI/Smart Triage/Potential Datasets/SkinBurns/'
#skf = StratifiedKFold(n_splits=2, shuffle=True)
#skf.get_n_splits(X, Y)
accuracy_list= list()
precision_list=list()
f1_list=list()
microprecision_list=list()
microf1_list=list()
means, mins, maxs = list(),list(),list()
f1means, f1mins, f1maxs = list(),list(),list()
microf1means, microf1mins, microf1maxs = list(),list(),list()
precmeans, precmins, precmaxs = list(),list(),list()
microprecmeans, microprecmins, microprecmaxs = list(),list(),list()

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,min_delta=.001,mode='auto')
foldNum=0
k=5
skf = StratifiedKFold(n_splits=k, shuffle=True)

for train_index, val_index in skf.split(X, Y):
             
       foldNum+=1
       model.compile(loss='categorical_crossentropy', optimizer='sgd',  metrics=['accuracy'] )
       print("Results for fold",foldNum)
       X_train, X_val = X[train_index], X[val_index]
       Y_train, Y_val = Y[train_index], Y[val_index]
       for eachindex in range(len(X_val)):
           shutil.move('/content/gdrive/MyDrive/MBZUAI/Smart Triage/Potential Datasets/SkinBurns/'+X_val[eachindex], 
                    'validation/'+X_val[eachindex])
       
       #Start ImageClassification Model
       train_datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True,  vertical_flip=True, brightness_range=[0.7,1.2], preprocessing_function=preprocess_input)

       validation_datagen = ImageDataGenerator( preprocessing_function=preprocess_input)
       train_generator = train_datagen.flow_from_directory(
            train_path,
            shuffle= True,
            target_size=IMAGE_SIZE,
            batch_size=32,
            subset='training')
       validation_generator = validation_datagen.flow_from_directory(
            validation_path,
            target_size=IMAGE_SIZE,
            batch_size=32,
            shuffle=False)  
       
       #history=
       model.fit(train_generator, 
                        epochs=40,
                        validation_data = validation_generator,
                        steps_per_epoch=int(len(X_train) // batch_size),
                        validation_steps=int(len(X_val) // batch_size),
                        callbacks=[callback])
       predictions = model.predict(validation_generator, verbose=1)
       #score = model.evaluate(validation_generator,batch_size =32)
       #print('eval', score)
       yPredictions = np.argmax(predictions, axis=1)
       true_classes = validation_generator.classes
       # evaluate validation performance
       #print("***Performance on Validation data***")
       accuracy=accuracy_score(true_classes, yPredictions)
       f1=f1_score(true_classes, yPredictions,average='macro')
       microf1= f1_score(true_classes, yPredictions,average='micro')
       precision= precision_score(true_classes, yPredictions,average='macro')
       microprecision= precision_score(true_classes, yPredictions,average='micro')
       #precision=precision_score(true_classes, yPredictions,average='weighted')
       #f1Score=f1_score(true_classes, yPredictions, average='weighted') 
       #print("Accuracy  : {}".format(accuracy))
       #print("Precision : {}".format(precision))
       #print("f1Score : {}".format(f1Score))
       #cm=confusion_matrix(true_classes, yPredictions)
       accuracy_list.append(accuracy)
       f1_list.append(f1)
       precision_list.append(precision)
       microf1_list.append(microf1)
       microprecision_list.append(microprecision)
       #print(cm)
       #valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)
       for returnindex in range(len(X_val)):
            shutil.move('validation/'+X_val[returnindex],'/content/gdrive/MyDrive/MBZUAI/Smart Triage/Potential Datasets/SkinBurns/'+X_val[returnindex])
#gotta change this to F1
k_mean=statistics.mean(accuracy_list)
k_max=max(accuracy_list)
k_min=min(accuracy_list)
means.append(k_mean)
mins.append(k_mean - k_min)
maxs.append(k_max - k_mean)
#==================
f1_mean=statistics.mean(f1_list)
f1_max=max(f1_list)
f1_min=min(f1_list)
f1means.append(f1_mean)
f1mins.append(f1_mean - f1_min)
f1maxs.append(f1_max - f1_mean)
#========================
prec_mean=statistics.mean(precision_list)
prec_max=max(precision_list)
prec_min=min(precision_list)
precmeans.append(prec_mean)
precmins.append(prec_mean - prec_min)
precmaxs.append(prec_max - prec_mean)
#==========================
microprec_mean=statistics.mean(microprecision_list)
microprec_max=max(microprecision_list)
microprec_min=min(microprecision_list)
microprecmeans.append(microprec_mean)
microprecmins.append(microprec_mean - microprec_min)
microprecmaxs.append(microprec_max - microprec_mean)
#===========================
microf1_mean=statistics.mean(microf1_list)
microf1_max=max(microf1_list)
microf1_min=min(microf1_list)
microf1means.append(microf1_mean)
microf1mins.append(microf1_mean - microf1_min)
microf1maxs.append(microf1_max - microf1_mean)

print('Image_size and batch_size')
print(IMAGE_SIZE,batch_size)
print('Minimum Accuracy')
print(mins)
print('Maximum Accuracy')
print(maxs)
print('Average Accuracy')
print(means)
#=============
print('Minimum macro precision')
print(precmins)
print('Maximum macro precision')
print(precmaxs)
print('Average macro precision')
print(precmeans)
#===============
print('Minimum macro F1')
print(f1mins)
print('Maximum macro F1')
print(f1maxs)
print('Average macro F1')
print(f1means)
#===============
print('Minimum micro precision')
print(microprecmins)
print('Maximum micro precision')
print(microprecmaxs)
print('Average micro precision')
print(microprecmeans)
#===============
print('Minimum micro F1')
print(microf1mins)
print('Maximum micro F1')
print(microf1maxs)
print('Average micro F1')
print(microf1means)
#plt.errorbar(folds, means, yerr=[mins, maxs], fmt='o')
# plot the ideal case in a separate color
#plt.plot(folds, [ideal for _ in range(len(folds))], color='r')
# show the plot
#plt.show()
   #model.fit_generator(train_generator, epochs=50,  verbose = 0, validation_data=test_generator)

   #scores = model.evaluate(X[test], Y[test], verbose=0)
   #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))




Results for fold 1
Found 3081 images belonging to 10 classes.
Found 772 images belonging to 10 classes.
Epoch 1/40
96/96 [==============================] - 1631s 17s/step - loss: 1.3606 - accuracy: 0.5536 - val_loss: 0.9498 - val_accuracy: 0.6419
Epoch 2/40
96/96 [==============================] - 78s 813ms/step - loss: 0.8666 - accuracy: 0.6655 - val_loss: 0.8894 - val_accuracy: 0.6654
Epoch 3/40
96/96 [==============================] - 78s 811ms/step - loss: 0.7447 - accuracy: 0.7156 - val_loss: 0.8150 - val_accuracy: 0.6836
Epoch 4/40
96/96 [==============================] - 78s 815ms/step - loss: 0.6869 - accuracy: 0.7402 - val_loss: 0.7621 - val_accuracy: 0.7031
Epoch 5/40
96/96 [==============================] - 78s 811ms/step - loss: 0.6384 - accuracy: 0.7557 - val_loss: 0.7388 - val_accuracy: 0.7070
Epoch 6/40
96/96 [==============================] - 78s 811ms/step - loss: 0.5946 - accuracy: 0.7707 - val_loss: 0.7628 - val_accuracy: 0.6966
Epoch 7/40
96/96 [====================

Image_size and batch_size
[250, 250] 32
Minimum Accuracy
[0.17112406777815015]
Maximum Accuracy
[0.07235556636901086]
Average Accuracy
[0.8939219952392901]
Minimum macro precision
[0.19680308404138358]
Maximum macro precision
[0.06899978513072047]
Average macro precision
[0.9033627351892217]
Minimum macro F1
[0.21613995690893129]
Maximum macro F1
[0.07684116164543553]
Average macro F1
[0.8905395817103611]
Minimum micro precision
[0.17112406777815015]
Maximum micro precision
[0.07235556636901086]
Average micro precision
[0.8939219952392901]
Minimum micro F1
[0.17112406777815015]
Maximum micro F1
[0.07235556636901086]
Average micro F1
[0.8939219952392901]
